# Code Snippet for CS 6140
***

## Before you continue
***
- This notebook only contains code snippet, not the full code for the assignment. I suppose I already cover the most difficult part :P


- Since I wrote my assignment in `Python` and converted them into `R` later, I'm afraid they may not as well performing as the `Python` version.


- Let's start!

## Asignment 2
***

### How to create *character* k-gram from a string?

You don't need to user the `tm` package to create k-gram representations. As far as I know, `tm` only works for "words", not "characters".

Actually, some simply `R` string indexing trick will suffice.

In [2]:
# You must have 'stringr' package installed
require('stringr')

In [1]:
# First create a string
s <- 'abcdefg'
s

[1] "abcdefg"

In [58]:
# Loop through every string to create k-gram 

# Set k
k <- 2

# First, create an empty list to store generated k-grams.
kgram <- c()
for (i in seq_len(str_length(s))) {
    word <- str_sub(s, i, i+k-1)
    if (str_length(word) == k) {
        kgram <- c(kgram, word)
    }
}
kgram

[1] "ab" "bc" "cd" "de" "ef" "fg"

### How to split string into words?

Suppose we have a string *"John starts his day with an angry look at his inbox"*, and we want to convert it into somthing like (if I understand your question correctly)

    ["John", "starts", ..., "inbox"]
    
`stringr::str_split()` is the way to go.

> In the previous version, I generate `words`(line 3) with `str_split(s, ' ')`, however, it actually returns a `list` instead of a `character` vector, which cause your problem. By indexing it with `[[1]]`, we returns a vector.

In [32]:
require(stringr)
s <- "John starts his day with an angry look at his inbox"
words <- str_split(s, ' ')[[1]] # You must add "[[1]]" here!
words

[1] "John"   "starts" "his"    "day"    "with"   "an"     "angry"  "look"  
 [9] "at"     "his"    "inbox"

### How to create *word* k-grams?

Very similar to what wo did in character k-gram.

In [33]:
# create a sample word list
doc <- c('w1', 'w2', 'w3', 'w4', 'w5')

# set k
k <- 2

# empty vector to store generated k-grams
kgram <- c()

# loop through each word in doc
for (i in seq_along(doc)) {
    word <- str_c(doc[i : (i+k-1)], collapse = ' ')
    if (i <= length(doc) - k) {
        kgram <- c(kgram, word)
    }   
}
kgram

[1] "w1 w2" "w2 w3" "w3 w4"

Actually, you don't need to intall package `stringr` to achieve that. `strsplit` in the "base" `R` does exactly the same thing. However, what's beautiful about `stringr` is that the naming of its functions ais very consistent and, I would say, elegent :P

### How do we build a min-hash signature? 

Suppose we want to build `t` hash functions then take the `min` of it, and we use the `sha1` function from the `openssl` package.

In [34]:
install.packages('openssl')

Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.5:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.5/PACKAGES'"

package 'openssl' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\rossz\AppData\Local\Temp\RtmpyIsdeh\downloaded_packages


For a string `s` like "AMD announces its earnning today". To get its hash signature, just run `sha1(s)`

In [35]:
library(openssl)
s <- "AMD announces its earnning today"
sha1(s)

[1] "d54101b36e17c5cbce3ee5f341555716785b5310"

If we want generate *multiple* hash functions, just add some "salt" (random string) to it. For example, for hash function `f1`, we use `sha1(str_c(s, 'string1'))`, for hash function `f2`, use `sha1(str_c(s, 'string2'))`. 

In [37]:
sha1(str_c(s, 'whatever-you-like'))

[1] "fb33276aad7d422758ed4a721a101a78ef2db53f"

The choice of salt is very arbitrary, you may use any string (or number, than convert it to string) you like.

Each random string will represent 1 hash function, that's why you have "t=10, 20, 30...". Effectively `t` is the number of hash functions, or, number of random salt.

### Given a string, how to build `t`=N hash functions? 

Suppose we have a document like `c('a b', 'b c', 'c d', 'e f')`, and we want to generate `t`=10 hash functions. 

In [22]:
s <- c('a b', 'b c', 'c d', 'e f') # 4 elements!

# we generate 10 random "salt", let's say they are 10 uniform numbers.
# don't forget to convert them into characters
salts <- as.character(runif(10))

# each salt will generate one hash fucntions
# and we use this hash function to hash "each" element in s, which will give us 4 values
# we then take the minimum of these 4 hash signatures.
minhash_list <- c()
for (i in seq_along(salts)) {
    hashed_s <- c()
    for (j in seq_along(s)) {
        hashed_s <- c(hashed_s, sha1(str_c(s[j], salts[i])))
    }
    # now there are 4 numbers in hashed_s
    # we take the min
    # this will be the minhash of s "given a particular salt" (here is salts[i])
    minhash <- min(hashed_s)
    
    # finally, we append minhash to minhash_list
    minhash_list <- c(minhash_list, minhash)
}
# Eventually there should be t=10 values in the minhash_list
print(minhash_list)

 [1] "115429024a15ffb9ab1577fe6934dc641d10be3c"
 [2] "00aada5e98e8351f959cef536c0e044969ec6344"
 [3] "5e09b7937bed66f98be1c1a980c9ad1711b35122"
 [4] "860bcdaa6b988dc8031503b57ab7e53e1c444035"
 [5] "301b8469746364d6a279dbf4fb55509bc22ede58"
 [6] "1d9345e5ed8f524bb6e1aa1b4311798a6c3c11d0"
 [7] "1fadfe064de5e356b573bd7bab34bd48341435b2"
 [8] "04230a41e073878b7871fcdae215b65b0a0b36d4"
 [9] "218eea02be8f0fafb4549a6300c22fa7ea445ba0"
[10] "3a41241638159fd5baf6f9f4df581cb4e1cc7e2d"


Repeat the previous steps on the second document, which will also give us a 10-element vector. Compute the fraction of similar elements, that will be the JS.

## Assignment 3

### Q1.B Report similarity

For example, suppose from Q1.A, your choice of `b` and `r` is 2 and 5, respectively, and you want to calculate pair (A,B) whose similarity is 0.75. All you need to do is plug them into the formula $f(s) = 1 - (1 - s^b)^r$, which is $1 - (1 - 0.75^2)^5$.

You even don't need to write code to compute that!

### Q2.A How to generate random unit vector

Suppose we want to generate two random unit vector with dimension of $d=10$.

First, generate two random normal distribution.

In [5]:
set.seed(42)
# u1 and u2 follow uniform distribution
# this is only for demonstration, you should use some loop to generate unit vector in batch!

d <- 10 # dimension
u1 <- runif(d/2) # 5 uniform
u2 <- runif(d/2) # 5 uniform

# genearte normal
y1 <- sqrt(-2 * log(u1)) * cos(2 * pi * u2) # 5 normal
y2 <- sqrt(-2 * log(u2)) * sin(2 * pi * u2) # 5 normal

# combine y1 and y2 to get a vector of dimemsion 10
y <- c(y1, y2)

# normalize
y <- y / sqrt(y^2)

Now you have succesfully generate a 10 dimension unit vector! Repeat it as many times as you wish

### Q2.B Plot CDF of inner product

This is quite trivial. 

### Q3.A Plot angular similarity

Still trivial, read into the vectors, normalize them, and then compute as required by the equation.

### Q3.B Plot angular similarity (with LSH)

Do as in Q3.A

## Assignment 4

Amrita's Sol

In [16]:
file.path <- "C:/users/rossz/OneDrive/Academy/the U/Assignment/DM-04-Clustering/data/C2.txt"

df <- read.table(file.path,header = F)
df
summary(df)

n <- 1040
#setting j=1 for all j belonging to N
j=1
for (i in 1:k) {
  M=0
  c[i]=x[i]
  for (j in 1:n){
    if (d(x[j],c[j])>M)
      M=d(x[j],c[j])
    else 
      c[i]=x[j]
    for (j in 1:n){
      if d(x[j],c[j]) > d(x[j], c[i]) then
      [j] = i
    }
  }
}

ERROR: Error in parse(text = x, srcfile = src): <text>:19:10: unexpected symbol
18:     for (j in 1:n){
19:       if d
             ^


## Assignment 5

Read in the text files

In [2]:
chars <- scan('../DM-05-Frequency/data/S1.txt', what = 'character') %>% str_split('') %>% unlist()
chars[1:5]

[1] "a" "c" "w" "b" "a"

This is the code for the Majority algorithm

In [ ]:
l <- list()
k <- 10

delete_zero_labels <- function(l) {
    for (name in names(l)) {
        if (l[[name]] <= 0) {
            l[[name]] <- NULL
        }
    }
    return(l)
}

for (i in 1:length(chars)) {
    char <- chars[i]
    
    # if char already in `l`, then add one
    if (char %in% names(l)) {
        l[[char]] <- l[[char]] + 1
    } 
    # if `l` is not full, then add char to `l`
    else if (length(l) < k-1) {
        l[[char]] <- 1
    }
    # otherwise, decrement all counters
    else {
        l <- lapply(l, `-`, 1)
        l <- delete_zero_labels(l)
    }
}
    
# print the result
print(l)